#                                       **CIFAR-10: Image Classification**

#   Step 1: Data Preprocessing & Loading 
##  Visualization of Images and Labels / Inserting Grayscale Conversion / Augmentation 


%pip install matplotlib
%pip install numpy
%pip install tensorflow
%pip install tensorflow-gpu

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns

In [2]:
from sklearn.model_selection import train_test_split
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, RandomFlip, RandomRotation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.utils import to_categorical

In [13]:
# Load the CIFAR-10 Dataset
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

In [ ]:
# Check data dimensions
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

In [ ]:
# Define a list with all the class labels for CIFAR-10
classes = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

# Function to visualize color images from CIFAR-10 dataset with correct labeling
def visualize_color_images_with_labels(images, labels, classes, images_per_class=10, title="CIFAR-10 Images"):
    num_classes = len(classes)
    total_images = num_classes * images_per_class

    plt.figure(figsize=(6, 6))
    image_count = 0

    # Loop through class labels to pick images_per_class images per class
    for class_index, class_name in enumerate(classes):
        class_images = images[labels.flatten() == class_index][:images_per_class]

        # Loop through the images, arranging them dynamically
        for img in class_images:
            plt.subplot(num_classes, images_per_class, image_count + 1)
            plt.imshow(img)
            plt.axis('off')
            
            # Add class label to the left side of each row
            if image_count % images_per_class == 0:
                plt.text(-30, 32 // 2, class_name, rotation=0, size='large', va='center', ha='right')
            
            image_count += 1
    
    plt.suptitle(title)
    plt.tight_layout()
    plt.show()

# Visualize color images from the CIFAR-10 training set
visualize_color_images_with_labels(x_train, y_train, classes, images_per_class=10, title="CIFAR-10 Training Images")


# Convert images to grayscale (Not in use because of DenseNet)

grayscale_x_train = tf.image.rgb_to_grayscale(x_train)
grayscale_x_test = tf.image.rgb_to_grayscale(x_test)

gray_x_train = np.array(grayscale_x_train)
gray_x_test = np.array(grayscale_x_test)

print(gray_x_train.shape)
print(gray_x_test.shape)

In [16]:
# Data Augmentation:



# Create augmentation layer for model (used further down)

data_augmentation = Sequential([
layers.RandomFlip("horizontal_and_vertical"),
layers.RandomRotation(0.2),
]) 


In [ ]:
# Normalize the images to the range [0, 1]
x_train_normalized = x_train.astype('float32') / 255.0
x_test_normalized = x_test.astype('float32') / 255.0

print(x_train_normalized.shape)
print(x_test_normalized.shape)

In [ ]:
from tensorflow.keras.utils import to_categorical

# One-hot encode the labels
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

print(y_train.shape)
print(y_test.shape)

Task, Diego:
Transfer Traning (VGG-16 can work well, imagenit, inseption, densnet, resnet) Check which one is the most efficient to clasify our image model.
Build a model Densnet
- Research different networks to see what kind of data they were trained on (image classes, how many...?)
- Decide on best one for our dataset
- Think about how many layers to add on top of that for our specific model
- Think about which layers to freeze/ unfreeze when training with the new layers
- Adjust epochs, other parameters related to our new model which could optimize

# DenseNet Model

In [ ]:
from keras.applications import DenseNet121

In [2]:
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Load the pre-trained DenseNet121 model
#base_model = DenseNet121(weights='imagenet', include_top=False)

# Load DenseNet121 with pre-trained ImageNet weights, excluding the top layer
base_model = DenseNet121(include_top=False, weights='imagenet', input_shape=(32, 32, 3), pooling='avg')

# Adding custom Top layers that will be trained for CIFAR-10 classification
# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully connected layer
x = Dense(128, activation='relu')(x)

# Add the output layer
predictions = Dense(10, activation='softmax')(x)

# Create the final model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define the data generators
train_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  vertical_flip=True,
                                  rotation_range=20)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(x_train_normalized, y_train, batch_size=32)
val_generator = val_datagen.flow(x_test_normalized, y_test, batch_size=32)

# Train the model
model.fit_generator(train_generator,
                  steps_per_epoch=100,
                  epochs=10,
                  validation_data=val_generator,
                  validation_steps=50)

# Use the model to make predictions
predictions = model.predict(x_test_normalized)


In [ ]:
# Load DenseNet121 with pre-trained ImageNet weights, excluding the top layer
base_model = DenseNet121(include_top=False, weights='imagenet', input_shape=(32, 32, 3), pooling='avg')

# Make the base model non-trainable
base_model.trainable = False

# Build the model
model = Sequential([
    base_model,
    Flatten(),
    Dense(1024, activation='relu'),
    BatchNormalization(),
    Activation('relu'),
    Dense(10, activation='softmax')  # CIFAR-10 has 10 classes
])

# Show model structure
model.summary()

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)
datagen.fit(x_train)

# Train the model
history = model.fit(datagen.flow(x_train, y_train, batch_size=64),
                    steps_per_epoch=len(x_train) / 64, epochs=10,
                    validation_data=(x_test, y_test))


    # Unfreeze the top 50 layers of the model
for layer in base_model.layers[-50:]:
    layer.trainable = True

# It's important to recompile the model after you make any changes to the 'trainable' attribute of any inner layer, so that your changes are taken into account
model.compile(optimizer=Adam(learning_rate=0.0001),  # Lower learning rate
              loss='categorical_crossentropy',
              metrics=['accuracy'])

import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, RandomFlip, RandomRotation, BatchNormalization, Activation
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator




#  Step 4: Model Evaluation
## Evaluate the Model and Compute Metrics